# Linked Data Representations and Nested Data Structures

## Preface: What is an Object?

In [ ]:
student1 = {"id": 90001, "name": "Alice"}
student2 = {"id": 90002, "name": "Bob"}
major1 = {"num": 42, "name": "Blah Blah Studies",
          "students": [student1, student2]}
print(student1["name"])
print(" and ".join(student["name"] for student in major1["students"]))

## [JSON](http://www.json.org/) representation of objects

In [ ]:
import json
j = json.dumps(major1)
j

In [ ]:
r = repr(major1)
r

In [ ]:
j == r.replace("'", '"')

In [ ]:
json.loads(j)

In [ ]:
eval(j) # don't do this unless you trust j!

### JSON in web APIs

[Here's an API](https://dog.ceo/dog-api/) that returns JSON:

In [ ]:
from urllib.request import urlopen
data = urlopen("https://dog.ceo/api/breeds/image/random").read()
data

In [ ]:
url = json.loads(data)["message"]
url

In [ ]:
from IPython.display import Image
Image(url=url)

* [MongoDB](https://www.mongodb.com/) supports storing a bunch of JSON "documents" and searching among them, with indexing (like we'll see below)
* Most other databases, like [MySQL](https://www.mysql.com/) or [PostgreSQL](https://www.postgresql.org/), are **relational**, which means they require data objects to be "flat", meaning that the value in each field is a basic type (number or string) instead of a recursive list or dictionary.

## Running Example

Let's consider the following scenario, motivated by [the web system](https://catsoop.org/website) that runs 6.009.  We want to represent the following interrelated pieces of data:

* **Students**, each with a name and kerberos
* **Psets**, each with a number and maximum number of points
* **Grades**, each assigning a number of points (score) to one student for a particular pset

## Representation 1: Linked Data with Linked Data Structures

In [ ]:
student1 = {"name": "Alice", "kerberos": "al1ce"}
student2 = {"name": "Bob", "kerberos": "b0b"}
pset1 = {"number": 1, "points": 10}
pset2 = {"number": 2, "points": 15}
grade1 = {"pset": pset1, "student": student1, "points": 9.5}
grade2 = {"pset": pset1, "student": student2, "points": 6}
grade3 = {"pset": pset2, "student": student2, "points": 12}
[grade1, grade2, grade3]

### Aliasing

In [ ]:
student2["name"] = "Robert"
grade1["pset"]["points"] = 11
[grade1, grade2, grade3]

### Interface and Implementation

In [ ]:
class LinkedRep:
    def __init__(self):
        pass
    def addStudent(self, name, kerberos):
        pass
        return student
    def addPset(self, number, points):
        pass
        return pset
    def addGrade(self, pset, student, points):
        pass
        return grade

In [ ]:
r = LinkedRep()
r.addStudent("Alice", "al1ce")
r.addStudent("Bob", "b0b")
r.addPset(1, 10)
r.addPset(2, 15)
#r.addGrade(pset with number 1, student with kerberos "al1ce", 9.5)??

In [ ]:
class LinkedRep(LinkedRep):
    def studentsWithName(self, name): # don't assume unique
        pass
    def studentWithKerberos(self, kerberos): # assume unique
        pass
    def psetWithNumber(self, number): # assume unique
        pass

In [ ]:
r = LinkedRep()
r.addStudent("Alice", "al1ce")
r.addStudent("Bob", "b0b")
r.addPset(1, 10)
r.addPset(2, 15)
r.addGrade(r.psetWithNumber(1), r.studentWithKerberos("al1ce"), 9.5)
r.addGrade(r.psetWithNumber(1), r.studentWithKerberos("b0b"), 6)
r.addGrade(r.psetWithNumber(2), r.studentWithKerberos("b0b"), 12)
r.grades

In [ ]:
def example(Rep):
    r = Rep()
    r.addStudent("Alice", "al1ce")
    r.addStudent("Bob", "b0b")
    r.addPset(1, 10)
    r.addPset(2, 15)
    r.addGrade(r.psetWithNumber(1), r.studentWithKerberos("al1ce"), 9.5)
    r.addGrade(r.psetWithNumber(1), r.studentWithKerberos("b0b"), 6)
    r.addGrade(r.psetWithNumber(2), r.studentWithKerberos("b0b"), 12)
    return r

## Representation 2: Linked Data with Unique IDs

### Serialization

In [ ]:
json.dumps([r.students,r.psets,r.grades])

In [ ]:
import pickle
pickle.dumps([r.students,r.psets,r.grades])

### Storing IDs instead of references/pointers

In [ ]:
student1 = {"id": 1, "name": "Alice", "kerberos": "al1ce"}
student2 = {"id": 2, "name": "Bob", "kerberos": "b0b"}
pset1 = {"id": 3, "number": 1, "points": 10}
pset2 = {"id": 4, "number": 2, "points": 15}
grade1 = {"id": 5, "pset": 3, "student": 1, "points": 9.5}
grade2 = {"id": 6, "pset": 3, "student": 2, "points": 6}
grade3 = {"id": 7, "pset": 4, "student": 2, "points": 12}
[grade1, grade2, grade3]

### Implementing the Interface

In [ ]:
class IDRep:
    def __init__(self):
        pass
    def addStudent(self, name, kerberos):
        pass
    def addPset(self, number, points):
        pass
    def addGrade(self, pset, student, points):
        pass
    def studentsWithName(self, name): # don't assume unique
        pass
    def studentWithKerberos(self, kerberos): # assume unique
        pass
    def psetWithNumber(self, number): # assume unique
        pass

In [ ]:
r = example(IDRep)
r.grades

In [ ]:
json.dumps(r.grades[0])

### Working with IDs and expansion

In [ ]:
class IDRep(IDRep):
    def studentWithID(self, id): # assume unique
        pass
    def psetWithID(self, id): # assume unique
        pass
    # for good measure:
    def gradeWithID(self, id): # assume unique
        pass
    def expandGrade(self, grade):
        pass

In [ ]:
r = example(IDRep)
g1 = r.grades[0]
g1

In [ ]:
r.studentWithID(g1["student"])

In [ ]:
e1 = r.expandGrade(g1)
e1

In [ ]:
e1["points"] = 10

In [ ]:
e1  # expanded copy

In [ ]:
g1  # database

## Speeding Up with Dictionary [Indexes](https://en.wikipedia.org/wiki/Database_index)

### Indexes in Linked Representation (1)

In [ ]:
class IndexedLinkedRep(LinkedRep):
    def __init__(self):
        super().__init__()
        pass
    def addStudent(self, name, kerberos):
        student = super().addStudent(name, kerberos)
        pass
        return student
    def addPset(self, number, points):
        pset = super().addPset(number, points)
        pass
        return pset
    def addGrade(self, pset, student, points):
        grade = super().addGrade(pset, student, points)
        pass
        return grade
    def studentsWithName(self, name):
        pass
    def studentWithKerberos(self, kerberos):
        pass
    def psetWithNumber(self, number):
        pass

In [ ]:
r = example(IndexedLinkedRep)
r.grades

### Indexes in ID Representation (2)

In [ ]:
class IndexedIDRep(IDRep):
    def __init__(self):
        super().__init__()
        pass
    def addStudent(self, name, kerberos):
        student = super().addStudent(name, kerberos)
        pass
        return student
    def addPset(self, number, points):
        pset = super().addPset(number, points)
        pass
        return pset
    def addGrade(self, pset, student, points):
        grade = super().addGrade(pset, student, points)
        pass
        return grade
    def studentsWithName(self, name): # don't assume unique
        pass
    def studentWithKerberos(self, kerberos): # assume unique
        pass
    def psetWithNumber(self, number): # assume unique
        pass
    def studentWithID(self, id): # assume unique
        pass
    def psetWithID(self, id): # assume unique
        pass
    def gradeWithID(self, id): # assume unique
        pass

In [ ]:
r = example(IndexedIDRep)
r.grades[0]

In [ ]:
r.studentsByName

In [ ]:
r.studentByKerberos

### Index Maintenance

In [ ]:
r.studentsWithName("Bob")

In [ ]:
r.studentWithKerberos("b0b")["name"] = "Robert"

In [ ]:
r.studentsWithName("Bob")

In [ ]:
r.studentsWithName("Robert")

In [ ]:
r = example(IDRep)
r.studentWithKerberos("b0b")["name"] = "Robert"
r.studentsWithName("Robert")

In [ ]:
class IndexedLinkedRep(IndexedLinkedRep):
    def changeStudentName(self, student, name):
        pass
    def changeStudentKerberos(self, student, kerberos):
        pass

In [ ]:
class IndexedIDRep(IndexedIDRep):
    def changeStudentName(self, student, name):
        pass
    def changeStudentKerberos(self, student, kerberos):
        pass

In [ ]:
for Rep in [IndexedLinkedRep, IndexedIDRep]:
    print(f"# {Rep.__name__}")
    r = example(Rep)
    r.changeStudentName(r.studentWithKerberos("b0b"), "Robert")
    print(r.studentsWithName("Robert"))

## Grade Queries

1. How many points did this student get on this pset?
2. What are all the grades for this student? (for displaying a summary)
3. How many points total did this student get on all psets?
4. What are all the grades on this pset? (for plotting or computing statistics)

In [ ]:
class LinkedRep(LinkedRep):
    def pointsForPsetAndStudent(self, pset, student): # assume unique
        pass
    def gradesForStudent(self, student):
        pass
    def totalPointsForStudent(self, student):
        pass
    def gradesForPset(self, pset):
        pass

class IDRep(IDRep):
    def pointsForPsetAndStudent(self, pset, student): # assume unique
        pass
    def gradesForStudent(self, student):
        pass
    def totalPointsForStudent(self, student): # same as above
        pass
    def gradesForPset(self, pset):
        pass

In [ ]:
for Rep in [LinkedRep, IDRep]:
    print(f"# {Rep.__name__}")
    r = example(LinkedRep)
    student = r.studentWithKerberos("b0b")
    print(f"{student['name']}'s grades: {r.gradesForStudent(student)}")
    print("Total point value:", r.totalPointsForStudent(student))

### Student + Pset Queries

In [ ]:
class IndexedIDRep1(IndexedIDRep):
    def __init__(self):
        super().__init__()
        pass
    def addGrade(self, pset, student, points):
        grade = super().addGrade(pset, student, points)
        pass
        return grade
    def pointsForPsetAndStudent(self, pset, student): # assume unique
        pass

In [ ]:
r = example(IndexedIDRep1)
r.pointsForPsetAndStudent(r.psetWithNumber(1),
                          r.studentWithKerberos("al1ce"))

In [ ]:
class IndexedLinkedRep1(IndexedLinkedRep):
    def __init__(self):
        super().__init__()
        pass
    def addGrade(self, pset, student, points):
        grade = super().addGrade(pset, student, points)
        pass
        return grade
    def pointsForPsetAndStudent(self, pset, student): # assume unique
        pass

In [ ]:
r = example(IndexedLinkedRep1)
r.pointsForPsetAndStudent(r.psetWithNumber(1),
                          r.studentWithKerberos("al1ce"))

In [ ]:
r.changeStudentName(r.studentWithKerberos("al1ce"), "Al-Ice")
r.pointsForPsetAndStudent(r.psetWithNumber(1),
                          r.studentWithKerberos("al1ce"))

### Grade List Queries

In [ ]:
class IndexedLinkedRep2(IndexedLinkedRep1):
    def __init__(self):
        super().__init__()
        pass
    def addGrade(self, pset, student, points):
        grade = super().addGrade(pset, student, points)
        pass
        return grade
    def gradesForStudent(self, student):
        pass
    def gradesForPset(self, pset):
        pass
    
class IndexedIDRep2(IndexedIDRep1):
    def __init__(self):
        super().__init__()
        pass
    def addGrade(self, pset, student, points):
        grade = super().addGrade(pset, student, points)
        pass
        return grade
    def gradesForStudent(self, student):
        pass
    def gradesForPset(self, pset):
        pass

In [ ]:
for Rep in [LinkedRep, IndexedLinkedRep2, IDRep, IndexedIDRep2]:
    print(f"# {Rep.__name__}")
    r = example(Rep)
    print(r.gradesForStudent(r.studentWithKerberos('b0b')))

In [ ]:
class NestedIndexedLinkedRep(IndexedLinkedRep1):
    def addStudent(self, name, kerberos):
        student = super().addStudent(name, kerberos)
        pass
        return student
    def addPset(self, number, points):
        pset = super().addPset(number, points)
        pass
        return pset
    def addGrade(self, pset, student, points):
        grade = super().addGrade(pset, student, points)
        pass
        return grade
    def gradesForStudent(self, student):
        pass
    def gradesForPset(self, pset):
        pass
    
class NestedIndexedIDRep(IndexedIDRep1):
    def addStudent(self, name, kerberos): # same
        student = super().addStudent(name, kerberos)
        pass
        return student
    def addPset(self, number, points): # same
        pset = super().addPset(number, points)
        pass
        return pset
    def addGrade(self, pset, student, points):
        grade = super().addGrade(pset, student, points)
        pass
        return grade
    def gradesForStudent(self, student):
        pass
    def gradesForPset(self, pset):
        pass

In [ ]:
for Rep in [LinkedRep, NestedIndexedLinkedRep, IDRep, NestedIndexedIDRep]:
    print(f"# {Rep.__name__}")
    r = example(Rep)
    print("BOB:", r.studentWithKerberos('b0b'))
    print("BOB's GRADES:", r.gradesForStudent(r.studentWithKerberos('b0b')))

In [ ]:
class AllNestedIndexedLinkedRep(IndexedLinkedRep):
    def addStudent(self, name, kerberos):
        student = super().addStudent(name, kerberos)
        pass
        return student
    def addPset(self, number, points):
        pset = super().addPset(number, points)
        pass
        return pset
    def addGrade(self, pset, student, points):
        grade = super().addGrade(pset, student, points)
        pass
        return grade
    def pointsForPsetAndStudent(self, pset, student):
        pass
    def gradesForStudent(self, student):
        pass
    def gradesForPset(self, pset):
        pass
    
class AllNestedIndexedIDRep(IndexedIDRep):
    def addStudent(self, name, kerberos): # same
        student = super().addStudent(name, kerberos)
        pass
        return student
    def addPset(self, number, points): # same
        pset = super().addPset(number, points)
        pass
        return pset
    def addGrade(self, pset, student, points):
        grade = super().addGrade(pset, student, points)
        pass
        return grade
    def pointsForPsetAndStudent(self, pset, student):
        pass
    def gradesForStudent(self, student):
        pass
    def gradesForPset(self, pset):
        pass

In [ ]:
for Rep in [NestedIndexedLinkedRep, AllNestedIndexedLinkedRep, NestedIndexedIDRep, AllNestedIndexedIDRep]:
    print(f"# {Rep.__name__}")
    r = example(Rep)
    print("BOB:", r.studentWithKerberos('b0b'))
    print("BOB's GRADES:", r.gradesForStudent(r.studentWithKerberos('b0b')))
    print("BOB's GRADE for PSET 1:", r.pointsForPsetAndStudent(
        r.psetWithNumber(1), r.studentWithKerberos('b0b')))

### Total Query

In [ ]:
class IndexedLinkedRep3(IndexedLinkedRep2):
    def __init__(self):
        super().__init__()
        pass
    def addStudent(self, name, kerberos):
        student = super().addStudent(name, kerberos)
        pass
        return student
    def addGrade(self, pset, student, points):
        grade = super().addGrade(pset, student, points)
        pass
        return grade
    def totalPointsForStudent(self, student):
        pass

class AllNestedIndexedLinkedRep3(AllNestedIndexedLinkedRep):
    def addStudent(self, name, kerberos):
        student = super().addStudent(name, kerberos)
        pass
        return student
    def addGrade(self, pset, student, points):
        grade = super().addGrade(pset, student, points)
        pass
        return grade
    def totalPointsForStudent(self, student):
        pass

In [ ]:
for Rep in [LinkedRep, IndexedLinkedRep3, AllNestedIndexedLinkedRep3]:
    print(f"# {Rep.__name__}")
    r = example(Rep)
    print("BOB:", r.studentWithKerberos('b0b'))
    print("BOB's TOTAL:", r.totalPointsForStudent(r.studentWithKerberos('b0b')))

## Another Example: Esports Data

Objects:

* **Players**: tag, name, country, birthdate, etc.
* **Games**: which players played, which characters those players chose, which map they played on, who won / how many points each player obtained, game duration, when/where the game happened

Queries:

* How many games has this player won, and how many have they lost?
* What are the top 10 players in terms of total games won?
* What are the top 10 players in terms of win/loss ratio?
* What is the ELO-like ladder ranking of this player?  What are the top 10 players?
* Which character chosen by players has the highest win/loss ratio?
  For example, in Starcraft, is it statistically best to choose Terran, Zerg, or Protoss?
* Same questions above for each game map
* Etc.

A real-world example of such a database for Starcraft II is [Aligulac](http://aligulac.com/), which provides [a downloadable database and database spec](http://aligulac.com/about/db/).  (There's also an [official API](https://develop.battle.net/documentation/api-reference/starcraft-2-community-api) based on JSON input/output.)  Aligulac's database is designed for the relational database system PostgreSQL, so each object is flat (values are primitive values, not lists or dictionaries).  Note how every object has an ID and links (e.g. from a match game to the two players that played the game) are implemented by storing IDs.

The database dump even specifies a collection of indexes to build.  For example, each match game is indexed by ID, game version, the ID of a period object (what season the game was played in), the ID of the first player object, the ID of the second player object, the race of the first player, the race of the second player, etc.  These indexes enable quick filtering of the entire list of games to subsets of interest.  When we ask the Aligulac website for [the top player rankings in the current period/season](http://aligulac.com/periods/latest/), it doesn't sift through all game records and compute it — rather, it shows what it has stored in its data structure, and whenever new games get played, it updates that data structure, like we did above.

For example, here's [the current data](http://aligulac.com/periods/latest/) for which race won the most games in Starcraft II during the period from October 10, 2019 to October 19, 2019:

Race    |vs.| Race   | Wins | Losses | Win Ratio
--------|---|--------|------|--------|----------
Protoss |vs.| Terran | 133  | 132    | 50.19%
Protoss	|vs.| Zerg   | 215  | 272    | 44.15%
Terran  |vs.| Zerg   | 160  | 186    | 46.24%